### Part 1: n-Gram Language Models

In [ ]:
# Import Libaries
import requests
import collections
import random
import math

In [ ]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
response.raise_for_status() # Raise an exception for invalid HTTP status codes
text_data = response.text

In [ ]:
# sample
random.seed(42)

pos = random.randint(0, len(text_data) - 1000)
print(text_data[pos:pos+100])

BY:
Many good morrows to my noble lord!

HASTINGS:
Good morrow, Catesby; you are early stirring
What


In [ ]:
# preprocessing - do not change
def preprocess_text(text_data):
  text_data = text_data.replace(',',' , ').replace(';', ' ').replace(':', ' ').replace('.',' . ').replace('?',' ? ').replace('!',' ! ')
  text_data = text_data.replace('-', ' ')
  text_data = text_data.replace('\'', '').replace('"', '')
  text_data = text_data.replace('  ', ' ')
  text_data = text_data.replace('\n\n','\n').replace('\n',' </s> <s> ')
  text_data = '<s> ' + text_data + ' </s>'
  text_data = text_data.lower()
  return text_data

text_data = preprocess_text(response.text)
print(f"Number of words: {len(text_data.split(' '))}")

Number of words: 328097


In [ ]:
train_data = text_data[:-10_000]
test_data = text_data[-10_000:]
len(train_data), len(test_data)

(1431030, 10000)

In [ ]:
vocab = set(train_data.split(' '))
print(f"Number of unique words: {len(vocab)}")
print(f"Sample unique words: {list(vocab)[:10]}")

Number of unique words: 12124
Sample unique words: ['', 'perdona', 'fosset', 'impossibilities', 'chronicle', 'induction', 'sward', 'garners', 'darts', 'apology']


 ## A. Dealing with Out of Vocabulary Words

In [ ]:
def identify_oov_words(corpus, n=3):
    """
    Identify out-of-vocabulary (OOV) words that appear less than `n` times in the dataset.

    Parameters:
    - dataset: The dataset to process. It should be a dictionary with a 'text' key.
    - n: The frequency threshold below which words are considered OOV.

    Returns:
    - A set of out-of-vocabulary words.
    """
    count = collections.Counter(corpus.split(' '))
    OOV = set()
    for word in count:
        if count[word]<3:
            OOV.add(word)
    return OOV



In [ ]:
oov_words = identify_oov_words(train_data)

vocab = vocab - oov_words
vocab.add('<UNK>')
print(f"Number of OOV words: {len(oov_words)}")
print(f"Expected number of OOV words: {7181}")

assert len(oov_words) == 7181

Number of OOV words: 7181
Expected number of OOV words: 7181


In [ ]:
train_data = ' '.join(['<UNK>' if word not in vocab else word for word in train_data.split(' ')])
test_data = ' '.join(['<UNK>' if word not in vocab else word for word in test_data.split(' ')])

## B. Create the N-Gram Models

In [ ]:
uni_counts = collections.defaultdict(lambda:0)
bi_counts = collections.defaultdict(lambda:0)
tri_counts = collections.defaultdict(lambda:0)
four_counts = collections.defaultdict(lambda:0)
five_counts = collections.defaultdict(lambda:0)

In [ ]:
def ngram_counts(corpus):
    data = corpus.split(' ')
    for i in range(len(data)):
        uni_counts[data[i]] += 1
        if i < (len(data) - 1):
            bi_counts[tuple(data[i:i+2])] += 1
        if i < (len(data) - 2):
            tri_counts[tuple(data[i:i+3])] += 1
        if i < (len(data) - 3):
            four_counts[tuple(data[i:i+4])] += 1
        if i < (len(data) - 4):
            five_counts[tuple(data[i:i+5])] += 1

ngram_counts(train_data)

In [ ]:
uni = collections.defaultdict(lambda:0)
bi = collections.defaultdict(lambda:0)
tri = collections.defaultdict(lambda:0)
four = collections.defaultdict(lambda:0)
five = collections.defaultdict(lambda:0)

In [ ]:
def compute_ngram_probabilities():

    for word, count in uni_counts.items():
        uni[word] = count / sum(uni_counts.values())

    for bigram, count in bi_counts.items():
        prev_word = bigram[0]
        bi[bigram] = count / uni_counts[prev_word]

    for trigram, count in tri_counts.items():
        prev_bigram = tuple(trigram[:2])
        tri[trigram] = count / bi_counts[prev_bigram]

    for fourgram, count in four_counts.items():
        prev_trigram = tuple(fourgram[:3])
        four[fourgram] = count / tri_counts[prev_trigram]

    for fivegram, count in five_counts.items():
        prev_fourgram = tuple(fivegram[:4])
        five[fivegram] = count / four_counts[prev_fourgram]

compute_ngram_probabilities()


In [ ]:
# # Evaluation
# assert five[('<s>', 'against', 'the', 'roman', 'state')] == 1.0 # prob of last given prev 4
# assert four[('remain', '</s>', '<s>', 'i')] == 0.25 # prob of last given prev 3
# assert tri[('did', 'see', 'and')] == 0.5 # prob of last given prev 2
# assert bi[('rash', 'like')] == 0.1 # prob of last given prev 1
# assert round(uni[('citizen')],5) == 0.00031 # prob of last

## C. Interpolation Smoothing

In [ ]:

def calculate_bigram_probability_with_smoothing(word1, word2):
    # INSERT CODE HERE
    bigram_count = bi_counts.get((word1, word2), 0)
    unigram_count = uni_counts.get(word1, 0)
    vocab_size = len(uni_counts)
    probability = (bigram_count + 1) / (unigram_count + vocab_size)

    return probability

## D. Evaluate Perplexity

In [ ]:
def compute_perplexity(data):
    """
    Computes the perplexity of a given text data using a bigram language model.

    Parameters:
    - data : str
    Returns:
    - float
    """

    assert len(data.split(' ')) >= 5
    words = data.split(' ')
    N = len(words)

    log_prob_sum = 0

    for i in range(N - 1):
        w1 = words[i]
        w2 = words[i+1]
        prob = calculate_bigram_probability_with_smoothing(w1, w2)
        log_prob_sum += math.log(prob)

    perplexity = math.exp(-log_prob_sum / (N))

    return perplexity



In [ ]:
assert round(compute_perplexity(test_data)) == 129

# Part 2. Logistic Regression



 In this question, you will be guided to implement logistic regression classifer from scratch. You will use LR classifer to do sentiment analysis task on Twitter dataset (the dataset is provided in the code).

## Import Data

In [ ]:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [ ]:
nltk.download('twitter_samples')
nltk.download('stopwords')

## Prepare the data


In [ ]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

* Train test split: 20% will be in the test set, and 80% in the training set.


In [ ]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [ ]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

## A.  Text processing






*   Remove old style retweet with 'RT' in the sentence
*   Remove hyperlinks

*   Remove hashtag
*   Tokenize the sentence using TweetTokenizer


*   Remove stop words
*   Use PorterStemmer to create stem of words in tweet












In [ ]:
stop_words = set(stopwords.words('english'))
def process_tweet(tweet):
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)

    tokenizer = TweetTokenizer(preserve_case=False)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweet_tokens = [word for word in tweet_tokens if word not in stop_words]

    stemmer = PorterStemmer()
    tweet_tokens = [stemmer.stem(word) for word in tweet_tokens]

    return tweet_tokens

We will create a function that will take tweets and their labels as input, go through every tweet, preprocess them, count the occurrence of every word in the data set and create a frequency dictionary.

Notice how the outer for loop goes through each tweet, and the inner for loop steps through each word in a tweet.
The freqs dictionary is the frequency dictionary that's being built.
The key is the tuple (word, label), such as ("happy",1) or ("happy",0). The value stored for each key is the count of how many times the word "happy" was associated with a positive label, or how many times "happy" was associated with a negative label.

In [ ]:
def build_freqs(tweets, ys):
    freqs = collections.defaultdict(int)

    for i, tweet in enumerate(tweets):
        processed_tweet = process_tweet(tweet)
        for word in processed_tweet:
            freqs[(word, int(ys[i]))] += 1

    return freqs


## B. Logistic regression


### Sigmoid



In [ ]:

def sigmoid(z):
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    return 1/(1 + np.exp(-z))

In [ ]:
def compute_cost(x, y, theta):
    m = len(y)
    z = np.dot(x, theta)
    h = sigmoid(z)

    J = -(1/m) * (np.dot(y.T, np.log(h)) + np.dot((1 - y).T, np.log(1 - h)))
    return J[0]

def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    m = len(y)
    epsilon = 1e-5
    J = 0

    for i in range(num_iters):
        h = sigmoid(np.dot(x, theta))
        error = h - y
        gradient = (1/m) * np.dot(x.T, error)
        theta = theta - alpha * gradient
        J = -(1/m) * (np.dot(y.T, np.log(h + epsilon)) + np.dot((1 - y).T, np.log(1 - h + epsilon)))
        # J_history.append(J)

        if i % 100 == 0:
            print(f"Iteration {i}, Cost: {J[0][0]}")

    return J, theta

## C. Extracting the features

* Given a list of tweets, extract the features and store them in a matrix. We will extract two features.
    * The first feature is the number of positive words in a tweet.
    * The second feature is the number of negative words in a tweet.
* Then train logistic regression classifier on these features.
* Test the classifier on a validation set.


In [ ]:
def extract_features(tweet, freqs):
    '''
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)

    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3))

    #bias term is set to 1
    x[0,0] = 1

    # write your code here
    pos_count = 0
    neg_count = 0

    for word in word_l:
        pos_count += freqs.get((word, 1.0), 0)
        neg_count += freqs.get((word, 0.0), 0)

    x[0, 1] = pos_count
    x[0, 2] = neg_count

    return x

## D. Training the Model

In [ ]:
def train_model(train_x, train_y, freqs, alpha=0.01, num_iters=1500):
    '''
    Input:
        train_x: a list of training tweets
        train_y: a numpy array of corresponding labels (1 = positive, 0 = negative)
        freqs: a frequency dictionary of (word, label) pairs
        alpha: learning rate
        num_iters: number of iterations to train the model
    Output:
        J: the final cost
        theta: the learned weight vector
    '''
    m = len(train_x)

    X = np.zeros((m, 3))

    for i in range(m):
        X[i, :] = extract_features(train_x[i], freqs)

    theta = np.zeros((3, 1))

    J, theta = gradientDescent(X, train_y, theta, alpha, num_iters)

    print(f"Final cost: {J}")
    print(f"Final weights: {theta}")

    return J, theta


## E. Testing your model

In [ ]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    '''
    # write your code here
    x = extract_features(tweet, freqs)
    z = np.dot(x, theta)
    y_pred = sigmoid(z)
    return y_pred

In [ ]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input:
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """

   # write your code here
    correct_predictions = 0

    for tweet, label in zip(test_x, test_y):
        y_pred = predict_tweet(tweet, freqs, theta)
        y_hat = 1 if y_pred > 0.5 else 0
        if y_hat == int(label):
            correct_predictions += 1

    accuracy = correct_predictions / len(test_x)

    return accuracy

In [ ]:
freqs = build_freqs(train_x, train_y)
J, theta = train_model(train_x, train_y, freqs, alpha=0.01, num_iters=1500)
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

<ipython-input-25-bcbc49081965>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  freqs[(word, int(ys[i]))] += 1


Iteration 0, Cost: 0.6931271807599442
Iteration 100, Cost: 0.17530401828043315


<ipython-input-26-edf42e1f19d6>:10: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-z))


Iteration 200, Cost: 0.17399426741432353
Iteration 300, Cost: 0.17520711117140692
Iteration 400, Cost: 0.17524481816504778
Iteration 500, Cost: 0.17519088355289383
Iteration 600, Cost: 0.1752999779349184
Iteration 700, Cost: 0.17494055392894578
Iteration 800, Cost: 0.17342522872189978
Iteration 900, Cost: 0.17543401586026822
Iteration 1000, Cost: 0.17568868255091752
Iteration 1100, Cost: 0.17526173459265482
Iteration 1200, Cost: 0.17539699226449101
Iteration 1300, Cost: 0.1731496904879286
Iteration 1400, Cost: 0.17522097825587518
Final cost: [[0.17517855]]
Final weights: [[  0.07790017]
 [  9.29876709]
 [-11.68433543]]


<ipython-input-31-b9eb3e7da36a>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if y_hat == int(label):


Logistic regression model's accuracy = 0.9855
